In [ ]:
import cv2 
import numpy as np 
import os
from matplotlib import pyplot as plt 
import mediapipe as mp 
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import pickle 
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import numpy as np
from sklearn.metrics import accuracy_score
from hmmlearn import hmm


In [ ]:
dir_path = "../MSDataSet" 
actions = np.array(os.listdir(dir_path))


results_path = os.path.join('ms_data_results')



for action in actions: 
    num_of_videos = os.listdir(dir_path+'/'+action)
    for video in range(len(num_of_videos)): 
        try: 
            os.makedirs(os.path.join(results_path, action, str(video)))
        except: 
            pass 


In [ ]:
label_map = {label: num for num, label in enumerate(actions)}

In [ ]:

missing_files = []
sequences, labels = [], []

for action in actions: 
    file_list = os.listdir(dir_path+"/"+action)

    for video in range(len(file_list)): 
        window = []
        filename = file_list[video]
        cap = cv2.VideoCapture(dir_path+"/"+action+"/"+filename)
        num_of_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        for frame_num in range(num_of_frames): 
            try: 
                res = np.load(os.path.join(results_path, action, str(video), "{}.npy".format(frame_num)))
                print(res)
                window.append(res)
                print("window")
                print(window)
            except FileNotFoundError: 
                missing_files.append(action + '/' + str(video)+ '/' +str(frame_num))
                print("Missing ")
                print(missing_files)
                continue
        sequences.append(window)
        labels.append(action)

In [ ]:
X = np.array(sequences)
Y= np.array(labels)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42 )

In [ ]:
y_train_encoded = np.array([label_map[label] for label in y_train])
y_test_encoded = np.array([label_map[label] for label in y_test])

In [ ]:
n_components = 3 
n_classes = len(np.unique(y_train_encoded))
models = [hmm.GaussianHMM(n_components=n_components, covariance_type='tied', n_iter=1000) for _ in range(n_classes)]

In [ ]:
for gesture_class in range(n_classes): 
    gesture_sequences = X_train[y_train_encoded == gesture_class]
    lengths = [len(seq) for seq in gesture_sequences]
    concatenated_sequences = np.concatenate(gesture_sequences)
    models[gesture_class].fit(concatenated_sequences,lengths)

In [ ]:
y_pred = []
for sequence in X_test: 
    likelihoods =[model.score(sequence) for model in models]
    y_pred.append(np.argmax(likelihoods))

In [ ]:

accuracy = accuracy_score(y_test_encoded, y_pred)
print(f"Classification accuracy: {accuracy}")

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test_encoded, y_pred))

In [ ]:
cm = confusion_matrix(y_test_encoded, y_pred)
print(cm)

In [ ]:
import matplotlib.pyplot as plt

import seaborn as sns

import numpy as np

class_names = ['Blue', 'Family', 'Happy', 'Man']

plt.figure(figsize=(10,7))

sns.heatmap(cm, annot=True, fmt='d', cmap='Oranges', xticklabels=class_names, yticklabels=class_names)

plt.title('Confusion Matrix')

plt.ylabel('True label')

plt.xlabel('Predicted label')

plt.show()

In [ ]:
with open("msaslhmm_models", "wb") as f: 
    pickle.dump(models,f)